In [5]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import scipy.interpolate
import os
import matplotlib.pyplot as plt
import scipy.interpolate
import matplotlib.gridspec as gridspec
import ctypes
from fatiando.gravmag import transform
import os.path
import matplotlib.image as mpimg

def findPeak(data,alpha): 
    """
    Load a txt file of a square window of magnetic anomaly data and plot the power spectrum related to a certain alpha.
    It should be used to visualize the appropriate interval to compute Zt and Z0.
    if a peak is not present, increase alpha and plot again.
    
    Parameters
    data (string, required) : file path for the 3-column txt  containing the magnetic anomaly data
    alpha (float, required) : defractal index
    """
    
    #Load magnetic anomaly data (3-column file, separated by coma)
    x,y,mag = np.loadtxt(data, usecols=(0,1,2), unpack = True, delimiter=',') 
    
    #Compute 2d arrays by gridding and interpolation of data values  
    N = int(np.sqrt(len(mag)))       
    shape = (N,N)
    x_grid = np.linspace(x.min(), x.max(), N)
    y_grid = np.linspace(y.min(), y.max(), N)
    xi,yi = np.meshgrid(x_grid,y_grid)
    zi = scipy.interpolate.griddata((x, y), mag,(xi,yi), method='cubic')
    
    #Calculate 2d power spectrum and radial average
    kx, ky, pds = transform.power_density_spectra(xi, yi, zi, shape) 
    k_radial = np.sqrt(kx*kx + ky*ky)
    pds_defractal = pds*(k_radial**alpha) #Defract the observed power density spectrum
    k_radial, pds_radial = transform.radial_average_spectrum(kx, ky, pds_defractal)
    k_radial = k_radial*1000 #Adjust unit from rad/m to rad/km
    
    #Plot radial average
    plt.plot(k_radial,np.log(pds_radial),'-o',c="k")
    plt.grid(lw=.5,c="lightblue")
    plt.title('alpha='+str(alpha))
    plt.xlabel("k (rad/km)")
    plt.ylabel("ln(Power)")
    plt.xlim(0,0.8) #Show values associated with K up to 0.8 rad/km
    
def defractal(data,Kzt_min,Kzt_max,Kz0_min,Kz0_max):
    """
    Compute the depth to the bottom of magnetic layer (DBML) from a magnectic anomaly data using the de-fractal method.
    Solutions for the DBML are calculated for various alpha values, incremented by 0.2 from 0 up to 4.
    All individual solutions are saved into individual .png files and all parameters are saved into a single .txt file.
    The Zb value associated with the lowest RMS misfit may indicate an ideal alpha value. We recommend to check the results
    of each alpha to choose the best visual matching.
    
    Parameters
    data (string, required) : file path for the 3-column txt  containing the magnetic anomaly data
    Kzt_min (float, required) : lowest K limit of the range used for the linear regression to compute Zt
    Kzt_max (float, required) : highest K limit of the range used for the linear regression to compute Zt
    Kz0_min (float, required) : lowest K limit of the range used for the linear regression to compute Z0
    Kz0_max (float, required) : highest K limit of the range used for the linear regression to compute Z0
    """
    #Load magnetic anomaly data (3-column file, separated by coma)
    x,y,mag = np.loadtxt(data, usecols=(0,1,2), unpack = True, delimiter=',') 
    
    #Compute 2d arrays by gridding and interpolation of data values    
    N = int(np.sqrt(len(mag)))       
    shape = (N,N)
    x_grid = np.linspace(x.min(), x.max(), N)
    y_grid = np.linspace(y.min(), y.max(), N)
    xi,yi = np.meshgrid(x_grid,y_grid)
    zi = scipy.interpolate.griddata((x, y), mag,(xi,yi), method='cubic')
    
    #Calculate 2d power spectrum and radial average for the centroid solution (alpha = 0)
    kx, ky, pds = transform.power_density_spectra(xi, yi, zi, shape) 
    k_radial_centroid, pds_centroid = transform.radial_average_spectrum(kx, ky, pds)
    k_radial_centroid = k_radial_centroid*1000
    k_radial_centroid = k_radial_centroid[np.where(k_radial_centroid<=0.8)] #Limits the usable K values up to 0.8 rad/km
    pds_centroid = pds_centroid[np.where(k_radial_centroid<=0.8)] #Limits the usable PDS values associated with K up to 0.8 rad/km
    
    #Set values for iteratvily computation of solutions
    alpha = 0
    a_inc = 0.2
    a_lim = 4
    
    #Calculate the central X and Y coordinate associated with Zb
    xmed = x.min()+(x.max()-x.min())/2
    ymed = y.min()+(y.max()-y.min())/2
    
    #Create empty lists to be filled with parameters from each individual calculated solution
    zts,zbs,alphas,rmss = [],[],[],[]
    
    #Create a txt file to receive parameters and write the first row as a header info (will overwirte if file already exists)
    with open('table_parameters.txt', "w") as arq:
        arq.write("X Y Zt Zb Alpha RMS\n")
    
    #Loop to compute Zb values for a specific alpha value untill it reaches the last tried alpha value
    while alpha <= a_lim+a_inc:
        
        #Calculate 2d power spectrum and radial average
        k_radial = np.sqrt(kx*kx + ky*ky)
        pds_defractal = pds*(k_radial**alpha) #Defract the observed power density spectrum
        k_radial, pds_radial = transform.radial_average_spectrum(kx, ky, pds_defractal)
        k_radial = k_radial*1000 #Adjust unit from rad/m to rad/km
        k_radial = k_radial[np.where(k_radial <= 0.8)] #Limits the usable K values up to 0.8 rad/km
        pds_radial = pds_radial[np.where(k_radial <= 0.8)] #Limits the usable PDS values associated with K up to 0.8 rad/km
        
        #Set the wanted range of values used for computing Zt by linear regression
        k_radial_indZt = np.where((k_radial >= Kzt_min) & (k_radial <= Kzt_max))
        k_radial_Zt=k_radial[k_radial_indZt]

        #Fit line using K ranges for Zt
        m, b1 = np.polyfit(k_radial_Zt, np.log(pds_radial)[k_radial_indZt], 1)
        zt = abs(m)/2
        
        #Set the wanted range of values used for computing Z0 by linear regression
        k_radial_indZ0 = np.where((k_radial >= Kz0_min) & (k_radial <= Kz0_max))
        k_radial_Z0=k_radial[k_radial_indZ0]
        
        #Fit line using K ranges for Z0
        m, b2 = np.polyfit(k_radial_Z0[1:], 
                np.log(np.sqrt(pds_radial)/k_radial)[k_radial_indZ0][1:], 1)
        z0 = abs(m)
        zb = 2*z0 - zt
        
        #Calculate RMS between calculated and modelled
        model = (np.log(np.exp(b1)*((1/np.exp(abs(k_radial)*zt)) - 1/np.exp(abs(k_radial)*zb))**2))
        soma = np.sum(np.log(((model[1:])-(pds_radial[1:]))**2))
        rms = np.sqrt(soma/len(model))
        
        #save figure with ln(power) centroid, ln(power) calculated, ln(power) modelled
        plt.plot(k_radial_centroid,np.log(pds_centroid),label = 'Centroid')
        plt.plot(k_radial,np.log(pds_radial),label = 'De-fractal')
        plt.plot(k_radial,model,label = 'Modelled')
        plt.legend()
        plt.xlim(0,0.8)
        plt.grid(lw=.5,c="lightblue")
        plt.title('alpha='+str(alpha))
        plt.xlabel("K (rad/km)")
        plt.ylabel("ln(Power)")
        plt.savefig("alpha%.1f.png"%alpha,dpi=300)   
        
        #Fill lists with the obtained parameters with the current solution
        zts.append(zt)
        zbs.append(zb)
        alphas.append(alpha)
        rmss.append(rms)
        
        print("Modelling for a = %.1f complete! Figure saved!"%alpha)
        
        #Increment the value of alpha by 0.2 and clear the plot figure
        alpha +=a_inc
        plt.clf()
        
    #Save txt with x,y,zt,zb,alpha,rms  
    with open('table_parameters.txt', "a") as arq:
        for i in range(len(zts)):
            arq.write("%.2f, %.2f, %.2f, %.2f, %.2f, %.2f\n"%(xmed,ymed,zts[i],zbs[i],alphas[i],rmss[i]))
    
    print("Done! Figures and table of parameters saved!")                          

In [ ]:
findPeak("data_example.txt",2.2)

In [ ]:
#Define the point position k Range for Zt and Z0
Kzt_min = 0.12
Kzt_max = 0.5
Kz0_min = 0
Kz0_max = 0.07

defractal("data_example.txt",Kzt_min,Kzt_max,Kz0_min,Kz0_max)